In [1]:
# @title Cài đặt các thư viện cần thiết
# Cài đặt thư viện xử lý video và thư viện AI
!pip install -q transformers diffusers accelerate scipy xformers omegaconf
!pip install -q git+https://github.com/huggingface/accelerate.git
!pip install -q Pillow Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 103.5 MB/s eta 0:00:00


In [3]:
!pip install av -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 24.4 MB/s eta 0:00:00


In [ ]:
# @title Chạy Stable Video Diffusion (SVD-XT)
import torch
from diffusers import StableVideoDiffusionPipeline
from PIL import Image
import io

# Cấu hình để chạy được trên T4 (fp16 và offload)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "stabilityai/stable-video-diffusion-img2vid-xt"

# Tải mô hình với tối ưu hóa
pipe = StableVideoDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16"
)
# Sử dụng offload để tiết kiệm VRAM, cần RAM CPU
pipe.enable_model_cpu_offload()

# --- Tải ảnh đầu vào mẫu ---
# Sử dụng một ảnh placeholder đơn giản.
# BẠN NÊN THAY THẾ BẰNG ẢNH CỦA MÌNH
print("Tải ảnh placeholder...")
!wget -q -O input_image.png https://giadinh.mediacdn.vn/296230595582509056/2024/1/4/hap-dan2-17043409388181947406930.jpg
image = Image.open("input_image.png").resize((1024, 576))
image.save("input_image_resized.png")
image = Image.open("input_image_resized.png")

# --- Tham số tạo video ---
# Motion Bucket ID càng cao, chuyển động càng mạnh
motion_bucket_id = 127
# FPS càng cao, video càng mượt
fps = 6
# Số bước lấy mẫu (ít bước thì nhanh hơn, chất lượng kém hơn)
num_inference_steps = 25

print("Đang tạo video SVD-XT. Quá trình này có thể mất vài phút...")
# Tạo video
video_frames = pipe(
    image,
    decode_chunk_size=8,
    num_frames=25,
    motion_bucket_id=motion_bucket_id,
    fps=fps,
    num_inference_steps=num_inference_steps,
    generator=torch.Generator(device=device).manual_seed(42)
).frames[0]

# --- Lưu và Hiển thị Video ---
import numpy as np
import av
from IPython.display import display, HTML

def save_video(frames, path, fps):
    """Lưu danh sách ảnh PIL thành file video .mp4"""
    width, height = frames[0].size
    container = av.open(path, mode='w')
    stream = container.add_stream('mpeg4', rate=fps)
    stream.width = width
    stream.height = height
    stream.pix_fmt = 'yuv420p'

    for img in frames:
        np_img = np.array(img)
        frame = av.VideoFrame.from_ndarray(np_img, format='rgb24')
        for packet in stream.encode(frame):
            container.mux(packet)

    for packet in stream.encode():
        container.mux(packet)
    container.close()

output_path = "svd_xt_output.mp4"
save_video(video_frames, output_path, fps)
print(f"Video đã lưu tại: {output_path}")

# Hiển thị video trong Colab
def display_video(path):
    video_html = f"""
    <video width="600" controls>
      <source src="{path}" type="video/mp4">
      Trình duyệt của bạn không hỗ trợ video HTML5.
    </video>
    """
    display(HTML(video_html))

display_video(output_path)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Tải ảnh placeholder...
Đang tạo video SVD-XT. Quá trình này có thể mất vài phút...


  0%|          | 0/25 [00:00<?, ?it/s]